In [2]:
isdefined(:PATH) || include("consts.jl")
using NIfTI, OnlineStats, KurchatovFMRI, Plots
using ImageView, GtkReactive, Colors, OffsetArrays
using ExcelReaders, ImageFiltering, Stats, Distributions, Makie
using ImageView, Images, Optim, HypothesisTests
pyplot()

"""
Exponential weighted Moving average of an array,
calculated online
"""
@inline function ewma(Y,λ = 0.3)
    s = Series(ExponentialWeight(λ), Mean())
    V = similar(Y)
    for (i,v) in enumerate(Y)
        fit!(s,v);
        V[i] = OnlineStats.value(s)[1]
    end
    V
end

"""
Delta-ball coordinates
"""
δ(ind::CartesianIndex, d=1) = CartesianRange(ind-d, ind+d)

#Helper functions for shapes
i2s(i) = ind2sub(SHAPE,i)
s2i(s) = sub2ind(SHAPE,s...)
s2i(s::CartesianIndex) = sub2ind(SHAPE,s.I...)

d = 3
D(ci) = [
    [ci+CartesianIndex((i,i,0)) for i = -d:d:d]...,
    [ci+CartesianIndex((0,i,i)) for i = -d:d:d]...,
    [ci+CartesianIndex((i,0,i)) for i = -d:d:d]...
]

"""
Mean of the correltaion matrix for delta-ball
around `cent` 
"""
function mn_cor(ni,cent, X = 1:400)    
    TS = reshape([ewma(ni[coord.I...,X], 0.1) for coord in δ(cent)], (27))   
    mean(cor(t1,t2) for t1 in TS, t2 in TS )
end

mn_cor

In [ ]:
subj = 1;
ni = niread("/media/ashedko/_data_/datasets/UNC/$subj/detrended.nii", mmap = true);

diff = open("out/dist_1_S.ser", "r") do f
  deserialize(f)
end
# maxs = [maximum(mat[:,i]) for i in 1:SIZE]
# maxs = reshape(maxs,SHAPE);Z = zeros(SHAPE)
kd_m = KData(PATH,"1/mean.nii","logs/1.*",
  x->x)
mn = kd_m.data

# sobel operator kernels for X Y Z
deltas = [Array(reshape(Kernel.sobel((true,true,true),x)[1], (3,3,3))) for x in 1:3]
G = zeros(SIZE) # Result of applying sobel to matrices of distances to the point
for i =1:SIZE
  Df = [sum(del.*reshape(diff[:,i], (3,3,3))) for del in deltas] # Offset array stuff is needed to match the shifted indextes 
  G[i] = sqrt(sum((x->x^2).(Df)))
end

Gl = reshape(G,SHAPE)
d = 1
Gclear = copy(Gl); # clearing eges of the frame
Gclear[:,:,1:2] .= 0 
Gclear[:,1:2,:] .= 0 
Gclear[1:2,:,:] .= 0 

Gclear[:,:,end-1:end] .= 0 
Gclear[:,end-1:end,:] .= 0 
Gclear[end-1:end,:,:] .= 0;


In [24]:
# OffsetArray(1:3,1:3,1:3)

1488.7514060436831

In [ ]:
# el,ind = findmax(Gclear)
# i2s(ind)
# [repr(x.I) for x in D(cent)]

histogram(filter(x->x != 0 && x<2000 ,reshape(Gclear,SIZE)),
    size=(800,600))

In [13]:
(CartesianIndex(SHAPE)-2), CartesianIndex((3,3,3))
rng = collect(CartesianRange(CartesianIndex((3,3,3)),(CartesianIndex(SHAPE)-2)))

87×105×87 Array{CartesianIndex{3},3}:
[:, :, 1] =
 CartesianIndex{3}((3, 3, 3))   …  CartesianIndex{3}((3, 107, 3)) 
 CartesianIndex{3}((4, 3, 3))      CartesianIndex{3}((4, 107, 3)) 
 CartesianIndex{3}((5, 3, 3))      CartesianIndex{3}((5, 107, 3)) 
 CartesianIndex{3}((6, 3, 3))      CartesianIndex{3}((6, 107, 3)) 
 CartesianIndex{3}((7, 3, 3))      CartesianIndex{3}((7, 107, 3)) 
 CartesianIndex{3}((8, 3, 3))   …  CartesianIndex{3}((8, 107, 3)) 
 CartesianIndex{3}((9, 3, 3))      CartesianIndex{3}((9, 107, 3)) 
 CartesianIndex{3}((10, 3, 3))     CartesianIndex{3}((10, 107, 3))
 CartesianIndex{3}((11, 3, 3))     CartesianIndex{3}((11, 107, 3))
 CartesianIndex{3}((12, 3, 3))     CartesianIndex{3}((12, 107, 3))
 CartesianIndex{3}((13, 3, 3))  …  CartesianIndex{3}((13, 107, 3))
 CartesianIndex{3}((14, 3, 3))     CartesianIndex{3}((14, 107, 3))
 CartesianIndex{3}((15, 3, 3))     CartesianIndex{3}((15, 107, 3))
 ⋮                              ⋱                                 
 CartesianIn

In [ ]:
q = quantile(reshape(Gclear,SIZE), 0.7)
msk = Gclear.> q # +quantile mask

F = falses(SHAPE) # No bounds mask
for i in CartesianRange(CartesianIndex((3,3,3)),(CartesianIndex(SHAPE)-2)) 
    F[i]= true
end

rng_pos_o = collect(CartesianRange(CartesianIndex((1,1,1)),(CartesianIndex(SHAPE))))
rng_pos = rng_pos_o[msk] 
@time res1 = [(cnt,mn_cor(ni,cnt)) for cnt in sample(rng_pos,4000)]
rng_neg = rng_pos_o[.!msk .& F]
@time res2 = [(cnt,mn_cor(ni,cnt)) for cnt in sample(rng_neg,4000)]
filter!(x->!isnan(x[2]),res1)
filter!(x->!isnan(x[2]),res2)
# sort!(res, by=x->x[2])
histogram((x->x[2]).(res1),label ="quantile +", alpha=.8, fillalpha=.8, bins = 64, title = "Local correlation")
histogram!((x->x[2]).(res2),label ="quantile -", alpha=.8, fillalpha=.8, bins = 64)
# mean((x->x[2]).(res))

In [ ]:
function zst(q)
    msk = Gclear.> q # +quantile mask
    rng_pos_o = collect(CartesianRange(CartesianIndex((1,1,1)),(CartesianIndex(SHAPE))))
    rng_pos = rng_pos_o[msk] 
    res1 = [(cnt,mn_cor(ni,cnt)) for cnt in sample(rng_pos,2000)]
    rng_neg = rng_pos_o[.!msk .& F]
    res2 = [(cnt,mn_cor(ni,cnt)) for cnt in sample(rng_neg,2000)]
    pvalue(UnequalVarianceTTest(res1,res2))
end

In [2]:
filter!(x->!isnan(x[2]),res1)
filter!(x->!isnan(x[2]),res2)
# sort!(res, by=x->x[2])
histogram((x->x[2]).(res2),label ="quantile -", alpha=.8, fillalpha=.8, bins = 64, title = "Local correlation")
histogram!((x->x[2]).(res1),label ="quantile +", alpha=.8, fillalpha=.8, bins = 64)
# mean((x->x[2]).(res))

LoadError: [91mUndefVarError: res1 not defined[39m

In [12]:
rng = collect(CartesianRange(CartesianIndex((3,3,3)),(CartesianIndex(SHAPE)-2)))
@time res = [(cnt,mn_cor(ni,cnt)) for cnt in sample(rng,4000)]
filter!(x->!isnan(x[2]),res)
sort!(res, by=x->x[2])
# plot((x->x[2]).(res))
mean((x->x[2]).(res))

 47.878894 seconds (142.07 M allocations: 5.290 GiB, 1.61% gc time)


0.65054184f0

In [71]:
# X = linspace(0,8π,1000)
# Y = randn(1000)/2. + cos.(X)/3

# Y = ni[28,28,57,:]
# Y2 = ni[26,26,55,:]
X = 1:3600
cent = CartesianIndex(45, 25, 31)
TS = reshape([ewma(ni[coord.I...,:], 0.1)[X] for coord in δ(cent)], (27))
# plot([ewma(Y2)[X],ewma(Y)[X]],labels=["[27,27,57,:]","[28,28,57,:]"])
p = plot(X,TS,labels=[repr(x.I) for x in δ(cent)])
m = [cor(t1,t2) for t1 in TS, t2 in TS ]
# mean(m)

# plot!(X,label=)
show(p)

Plot{Plots.PlotlyBackend() n=27}

In [170]:
m = [cor(t1,t2) for t1 in TS, t2 in TS ]
mean(m)
ticks = [repr(x.I) for x in δ(cent)]
# show(ticks)
spy(m,xformatter = n -> ticks[Int(n)],size = (600,600))

In [83]:
r = randn(100)
plot(r)
plot!(ewma(r,0.3))

In [100]:
d = 1
Gclear = copy(Gl);
Gclear[:,:,1:2] .= 0 
Gclear[:,1:2,:] .= 0 
Gclear[1:2,:,:] .= 0 

Gclear[:,:,end-1:end] .= 0 
Gclear[:,end-1:end,:] .= 0 
Gclear[end-1:end,:,:] .= 0;


In [97]:
plot([maximum(Gclear[:,:,x]) for x=1:size(Gl,3)])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 0 
 
 
 5000 
 
 
 10000 
 
 
 15000 
 
 
 20000 
 
<polyline clip-path="url(#clip4802)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 48.6962,375.813 54.6031,375.813 60.5099,337.595 66.4168,333.39 72.3237,332.408 78.2306,326.162 84.1374,329.878 90.0443,336.495 95.9512,333.559 101.858,337.61 
 107.765,337.906 113.672,347.233 119.579,343.866 125.486,348.47 131.392,338.038 137.299,333.294 143.206,338.213 149.113,341.034 155.02,339.28 160.927,338.663 
 166.834,330.091 172.741,325.874 178.647,330.197 184.554,313.869 190.461,280.732 196.368,235.045 202.275,230.735 208.182,147.059 214.089,37.6863 219.996,14.0629 
 225.902,11.811 231.809,140.004 237.716,87.2374 243.623,118.062 249.53,265.885 255.437,271.534 261.344,285.76 267.251,264.66 273.157,294.342 279.064,301.185 
 284.971,309.82 290.878,307.311 296.785,305.169 302.692,312.086 308.599,320.884 314.506,332.743 320.412,325.062 326.319,339.437 332.226,339.558 338.133,327.794 
 344.04,327.569 349.947,328.423 355.854,319.6 361.761,322.208 367.667,331.126 373.574,334.577 379.481,340.065 385.388,341.486 391.295,339.561 397.202,348.016 
 403.109,340.913 409.016,340.475 414.922,336.369 420.829,323.131 426.736,313.558 432.643,314.169 438.55,304.01 444.457,294.688 450.364,281.782 456.271,291.216 
 462.177,308.163 468.084,301.835 473.991,277.55 479.898,290.469 485.805,322.89 491.712,319.892 497.619,326.824 503.526,331.043 509.432,338.524 515.339,355.892 
 521.246,362.14 527.153,361.435 533.06,362.196 538.967,362.111 544.874,361.319 550.781,363.209 556.687,366.504 562.594,368.825 568.501,369.302 574.408,375.813 
 580.315,375.813 
 "/>
 
 
 
 
 y1

search: revise Revise invisible glvisualize SupervisedLoss UnsupervisedLoss

Couldn't find vis
Perhaps you meant cis, is, view, vox, var, vec, svds, div, fps, aic, bic or fit


No documentation found.

Binding `vis` does not exist.
